<a href="https://colab.research.google.com/github/ricardomgi/elevation_data/blob/main/Elevation_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import time

### **Open Elevetaion API**

Documentation: https://open-elevation.com

In [ ]:
# script for returning elevation from lat, long, based on open elevation data
# which in turn is based on SRTM
def get_elevation_oea(lat, long):
    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations={lat},{long}')
    r = requests.get(query).json()  # json object, various ways you can extract value
    # Only get the json response in case of 200 or 201
    if r.status_code == 200 or r.status_code == 201:
      # one approach is to use pandas json functionality:
      elevation = pd.json_normalize(r, 'results')['elevation'].values[0]
    else: 
      elevation = None
    return elevation

def get_elevation_oea_multiple(locs):
    url = 'https://api.open-elevation.com/api/v1/lookup'
    coords_data = ''
    for x in range(len(locs)):
      coordinates = f'"latitude":{list(zip(*locs))[0][x]},"longitude":{list(zip(*locs))[1][x]}'
      coords_data = coords_data + '{' + coordinates + '},'
    coords_data = coords_data[:-1]
    coords_data = '{"locations": [' + coords_data + ']}'
    r = requests.post(url, json=coords_data)
    # Only get the json response in case of 200 or 201
    if r.status_code == 200 or r.status_code == 201:
      parsed = json.loads(r.text)
      # Extract elevations from result
      elevation = [i['elevation'] for i in parsed['results']]
    else: 
        elevation = None
    return elevation

### **Open Topo Data**

Documentation: https://www.opentopodata.org

In [ ]:
# script for returning elevation from lat, long, based on Open Topo Data
# Datasets: mapzen, aster30m, srtm30m, srtm90m, etopo1
# Resolution: ~30 m
# Extent: Global
def get_elevation_otd(lat, long, dataset='mapzen'):
    query = ('https://api.opentopodata.org/v1/'
             f'{dataset}?locations={lat},{long}')
    r = requests.get(query).json()  # json object, various ways you can extract value
    # one approach is to use pandas json functionality:
    elevation = pd.json_normalize(r, 'results')['elevation'].values[0]
    return elevation

# script for returning elevation from lat, long, based on Open Topo Data
# Datasets: mapzen, aster30m, srtm30m, srtm90m, etopo1
# Resolution: ~30 m
# Extent: Global
def get_elevation_otd_multiple(locs, dataset='mapzen'):
    url = 'https://api.opentopodata.org/v1/' + dataset
    data = {"locations": locs}
    response = requests.post(url, data=data).json() # json object, various ways you can extract value
    # one approach is to use pandas json functionality:
    elevation = pd.json_normalize(response, 'results')['elevation'].values
    elevation = elevation.astype(int)
    return elevation

### **Google Elevation API**

Documentation: https://developers.google.com/maps/documentation/elevation/overview

In [ ]:
# Configure API request link
def get_elevation_gapi(lat, long):
  key_str = "API_key"
  url = "https://maps.googleapis.com/maps/api/elevation/json"
  # Make request, parse results
  r = requests.get(url+"?locations="+str(lat)+","+str(long)+"&key="+key_str).json()
  # one approach is to use pandas json functionality:
  elevation = pd.json_normalize(r, 'results')['elevation'].values[0]
  return round(elevation, 2)

def get_elevation_gapi_multiple(locs):
  url_root = "https://maps.googleapis.com/maps/api/elevation"
  key_str = "API_key"
  output_fmt = "json"
  request_str = "%s/%s?locations=%s&key=%s"
  # Make request, parse results
  r = requests.get(request_str % (url_root,output_fmt,locs,key_str))
  parsed = json.loads(r.text)
  # Extract elevations from result
  elevation = [i['elevation'] for i in parsed['results']]
  elevation = list(np.around(np.array(elevation),2))
  return elevation

### **Get the elevation data at one (latitude,longitude) point:**

Can be used to obtain the elevation of a single point

In [ ]:
print(get_elevation_oea(4.7967, -75.7923))
print(get_elevation_otd(4.7967, -75.7923, 'mapzen'))
print(get_elevation_gapi(4.7967, -75.7923))

### **Get the elevations of multiple locations:**

Can be used to get the elevation of multiple coordinates. If you plan on making large requests, consider using the POST api.

**Open Elevation API Note:** The GET API is limited to 1024 bytes in the request line.

**Google API Note:** URLs must be properly encoded to be valid and are limited to 8192 characters for all web services. Be aware of this limit when constructing your URLs. Note that different browsers, proxies, and servers may have different URL character limits as well.

In [ ]:
# Function to concat the coordinates
def locations(lat, long):
  locs=''
  for x in tuple(zip(lat, long)):
    locs += str(x[0]) + ',' + str(x[1]) + '|'
  locs = locs[:-1]
  return locs

list_coords = [(6.17593, -75.6260), (6.17593, -75.6260), (5.92688, -75.6758)]
lcs = locations(list(zip(*list_coords))[0], list(zip(*list_coords))[1])
print(get_elevation_oea_multiple(coords))
print(get_elevation_otd_multiple(lcs))
print(get_elevation_gapi_multiple(lcs))

### **Function to get the elevation data for multiple points**

Any of the mentioned APIs can be used as an argument:

*   Open Elevation API -> **oea**
*   Open Topo Data API -> **otd**
*   Google Elevation API -> **gapi**

The quotas used for each API are the following according to their documentation:

**Open Elevation API:**
*   The GET API is limited to 1024 bytes in the request line
*   The POST API currently has no limit

**Open topo Data API:**
*   Max **100** locations per request
*   Max **1** call per second
*   Max **1000** calls per day

**Google Elevation API**:
*   **512** locations per request
*   **100** requests per second (QPS)
*   No longer limited to a maximum number of requests per day (QPD)




In [ ]:
def elevation(coords, api, otd_dataset='mapzen'):
  if api == 'oea':
    request = 1000
  elif api == 'otd':
    request = 100
  elif api == 'gapi':
    request = 512
  elev = []
  iterations = int(len(coords)/request) + 1
  for x in range(iterations):
      iter_str = coords[x*request:(x+1)*request]
      #Build concatenated string for API request
      locs = '|'.join(iter_str)
      #Make request, parse results
      if api == 'oea':
        elev = elev + get_elevation_oea_multiple(coords)
      elif api == 'otd':
        elev = elev + list(get_elevation_otd_multiple(locs, otd_dataset))
      elif api == 'gapi':
        elev = elev + get_elevation_gapi_multiple(locs)
      #Slow down API requests so we aren't blocked
      time.sleep(1)
  return elev

### **Get de elevation data from multiple point in dataframe Latitude and Longitude colummns**

In [ ]:
data = pd.read_csv('Data_test.csv', sep=';')
data = data[pd.to_numeric(data["Latitude"], errors='coerce').notnull()]
data = data[pd.to_numeric(data["Longitude"], errors='coerce').notnull()]

# Concat latitude and longitude separated by comma
list_coords = list(data.Latitude.astype(str) + ',' + data.Longitude.astype(str))

# Get de elevation data from APIs
data['Elevation_OTD'] = elevation(list_coords, 'otd')
data['Elevation_GAPI'] = elevation(list_coords, 'gapi')

print(data)